In [2]:
import numpy as np

In [45]:
class FastGradMethod:
    def __init__(self, gamma, n, epsilon):
        self.alpha = 0
        self.a     = 0
        
        # constants
        self.epsilon = epsilon
        self.gamma = gamma
        self.l     = 1 / gamma
        self.n     = n
        
        self.u_lambda = np.ones(n) / n**2
        self.u_mu     = np.ones(n) / n**2
         
        self.y_lambda = np.zeros(n)
        self.y_mu     = np.zeros(n)
        
        self.x_lambda   = np.zeros([n])
        self.x_mu       = np.zeros([n])
        
        self.x_0 = np.ones([n, n]) / (n**2)
        
        self.alpha_sum = 0
        # x(y_lambda, y_mu)
        self.x_sum = np.zeros([n, n])
        
    def x_hat(self, c):
        n = self.n
        a = self.x_0 * np.exp(
            -(self.gamma + c + self.y_lambda.repeat(n).reshape(-1, n) + self.y_mu.repeat(n).reshape(-1, n)) / self.gamma
        )
        return a / a.sum()
    
    def x_wave(self, c):
        return 1 / self.a * self.alpha_sum * self.x_sum
    
    def __new_alpha(self):
        return 1 / (2 * self.l) + np.sqrt(1 / (4 * (self.l**2)) + self.alpha**2)
    
    def __new_a(self, new_alpha):
        return self.a + new_alpha
    
    def __new_y(self):
        new_alpha = self.__new_alpha()
        return (self.__new_alpha() * self.u_lambda + self.a * self.x_lambda) / self.__new_a(new_alpha),\
               (self.__new_alpha() * self.u_mu + self.a * self.x_mu) / self.__new_a(self.__new_alpha())
    
    def __new_u(self):
        x_hat = self.x_hat(c)
        return self.u_lambda - self.alpha * (p - x_hat.sum(1)),\
               self.u_mu - self.alpha * (q - x_hat.sum(0))
        
    def __new_x(self):
        new_alpha = self.__new_alpha()
        return (self.alpha * self.u_lambda + self.a * self.x_lambda) / self.__new_a(new_alpha),\
               (self.alpha * self.u_mu + self.a * self.x_mu) / self.__new_a(self.__new_alpha())
    
    def f(self, c, x_cap):
        return np.sum(c * x_cap) + self.gamma * np.sum(x_cap * np.log(x_cap / self.x_0))
        
    def phi(self, c, lambda_x, mu_x):
        n = self.n
        return -np.sum(lambda_x * p) - np.sum(mu_x * q) +\
        self.gamma * np.log(1 / np.exp(1) * np.sum(
            self.x_0 * np.exp(
                -(self.gamma + c + lambda_x.repeat(n).reshape(-1, n) + mu_x.repeat(n).reshape(-1, n).T)
            )
        ))
    
    def fit(self, c, p, q):
        k = 0
        while True:
            k+=1
            
            self.alpha_sum += self.alpha
            self.x_sum += self.x_hat(c)
            
            self.y_lambda, self.y_mu = self.__new_y()
            self.u_lambda, self.u_mu = self.__new_u()
            self.x_lambda, self.x_mu = self.__new_x()
            
            self.alpha = self.__new_alpha()
            self.a     = self.__new_a(self.alpha)
            
            x_wave = self.x_wave(c)
            r = self.epsilon / np.sqrt((self.x_lambda**2).sum() + (self.x_mu**2).sum())
            
            criteria_a = np.sqrt(np.sum((x_wave.sum(1) - p)**2) + np.sum((x_wave.sum(0) - 1)**2)) <= r
            criteria_b = self.f(c, x_wave) - self.phi(c, p, q) <= self.epsilon
            
            if criteria_a and criteria_b:
                return k

In [46]:
fgrad = FastGradMethod(0.3, 5, 0.001)

In [47]:
def sample_batch(n):
    C = np.random.uniform(0, 10, size=[n, n])
    p = np.random.dirichlet(np.ones(5), size=1).ravel()
    q = np.random.dirichlet(np.ones(5), size=1).ravel()
    return C, p, q

In [48]:
c, p, q = sample_batch(5)

In [49]:
fgrad.fit(c,p,q)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:82: RuntimeWarning: divide by zero encountered in double_scalars
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in log
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in multiply
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in exp
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


KeyboardInterrupt: 